In [1]:
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Constants
beta_0 = 2.0
beta_1 = 3.0
n_samples = 100
sigma = 1.0  # Standard deviation of the noise

# Generate synthetic data
X = 10 * np.random.rand(n_samples)
epsilon = np.random.normal(0, sigma, n_samples)
Y = beta_0 + beta_1 * X + epsilon

In [2]:
from scipy.stats import norm, invgamma

# Prior distributions
def prior_beta_0():
    return norm(0, 10).rvs()

def prior_beta_1():
    return norm(0, 10).rvs()

def prior_sigma2():
    return invgamma(1).rvs()

In [3]:
def likelihood(X, Y, beta_0, beta_1, sigma2):
    Y_pred = beta_0 + beta_1 * X
    residuals = Y - Y_pred
    return np.prod(norm(loc=0, scale=np.sqrt(sigma2)).pdf(residuals))

In [4]:
def monte_carlo_posterior(X, Y, num_samples=10000):
    samples = []
    for _ in range(num_samples):
        b0 = prior_beta_0()
        b1 = prior_beta_1()
        s2 = prior_sigma2()
        post = likelihood(X, Y, b0, b1, s2) * norm(0, 10).pdf(b0) * norm(0, 10).pdf(b1) * invgamma(1).pdf(s2)
        samples.append((b0, b1, s2, post))
    
    # Normalize the posterior probability (for simplicity, consider it unnormalized here)
    return samples

# Generate samples from the posterior
posterior_samples = monte_carlo_posterior(X, Y, num_samples=10000)

In [5]:
# Extracting samples and weights
beta_0_samples = [s[0] for s in posterior_samples]
beta_1_samples = [s[1] for s in posterior_samples]
sigma2_samples = [s[2] for s in posterior_samples]
weights = [s[3] for s in posterior_samples]

# Compute weighted averages
beta_0_est = np.average(beta_0_samples, weights=weights)
beta_1_est = np.average(beta_1_samples, weights=weights)
sigma2_est = np.average(sigma2_samples, weights=weights)

print(f"Estimated beta_0: {beta_0_est}")
print(f"Estimated beta_1: {beta_1_est}")
print(f"Estimated sigma^2: {sigma2_est}")

Estimated beta_0: 2.166892014785945
Estimated beta_1: 2.9566554039577913
Estimated sigma^2: 0.8220087340441655
